In [99]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pymysql 

In [100]:
def get_columns(info_COLUMNS):
    lst=[]
    for i in info_COLUMNS:
        lst.append(i[0])
    return lst

In [101]:
# 用pymysql连接尝试
# 连接数据库
conn = pymysql.connect(
	host = '123.60.11.177',
	port = 3306,
	user = 'root',
	password = 'ncu@jw114',
	db = 'examArrange1',
	charset='utf8'
	)
cur = conn.cursor()
cur.execute("SELECT VERSION()")
version = cur.fetchall()
cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'classroom_inf') AND (ordinal_position >= 1) order by ordinal_position")
classroom_inf_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM classroom_inf")
classroom_inf=pd.DataFrame(cur.fetchall(),columns=classroom_inf_columns)

cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'classs_inf') AND (ordinal_position >= 1) order by ordinal_position")
classs_inf_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM classs_inf")
classs_inf=pd.DataFrame(cur.fetchall(),columns=classs_inf_columns)

cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'teacher_inf') AND (ordinal_position >= 1) order by ordinal_position")               
teacher_inf_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM teacher_inf")
teacher_inf=pd.DataFrame(cur.fetchall(),columns=teacher_inf_columns)

cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'mid_testinfo') AND (ordinal_position >= 1) order by ordinal_position")
mid_testinfo_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM mid_testinfo")
mid_testinfo=pd.DataFrame(cur.fetchall(),columns=mid_testinfo_columns)

cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'dm_classroom_state') AND (ordinal_position >= 1) order by ordinal_position")
dm_classroom_state_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM dm_classroom_state")
dm_classroom_state=pd.DataFrame(cur.fetchall(),columns=dm_classroom_state_columns)
#关闭数据库连接
conn.close()


In [102]:
dm_classroom_state

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,1,慧源楼101,前湖校区,1,1,None
1,2,慧源楼101,前湖校区,2,1,None
2,3,慧源楼101,前湖校区,3,1,None
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2443,2444,虚拟教室01,前湖校区,20,1,None
2444,2445,虚拟教室01,前湖校区,21,1,None
2445,2446,虚拟教室01,前湖校区,22,1,None
2446,2447,虚拟教室01,前湖校区,23,1,None


# 选出公共课

In [103]:
#放回当前学科当前班级对应的班级编号
#输入为一个temp（学姐设计的第一个表）series --与apply使用输入为dataframe
#输出对应的课的班级编号
def get_ci_nation_no(row):
    x=classs_inf.ci_notice_no[(classs_inf.ci_course_no==row[0])&(classs_inf.ci_class_name==row[2])]
    return x.values[0]
#输入cl_notice_no每班级编号，为int
#输出为当前班级对于的人数
def get_class_student_number(cl_notice_no):
    return classs_inf[classs_inf.ci_notice_no==cl_notice_no].ci_student_number.values

In [104]:
#教室时间对
D=dm_classroom_state[dm_classroom_state.cr_name!='虚拟教室01'] #维护的一个笛卡儿排课表
dm_classroom_state[dm_classroom_state.cr_name!='虚拟教室01']
new_D=D[:]
#课表时间对
temp=pd.read_csv("./temp.csv")
ci_nation_no=temp.apply(get_ci_nation_no,axis=1).values
temp.insert(0,"ci_nation_no",ci_nation_no)

# 两个输入数据

In [105]:
new_D #0是序号--后面可以用来存班级编号，
      #1位教室名称，2对应校区，3对应考试时间，4对应的是否能够使用，5对应是否安排考场

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,1,慧源楼101,前湖校区,1,1,None
1,2,慧源楼101,前湖校区,2,1,None
2,3,慧源楼101,前湖校区,3,1,None
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2419,2420,慧源楼520,前湖校区,20,1,None
2420,2421,慧源楼520,前湖校区,21,1,None
2421,2422,慧源楼520,前湖校区,22,1,None
2422,2423,慧源楼520,前湖校区,23,1,None


In [106]:
temp #ci_nation_no班级编号，ci_course_no	课程名称（可不用），
     #test_time需要安排的考试时间，ci_class_name班级名称（可不用）

,ci_nation_no,ci_course_no,test_time,ci_class_name
0,202020212000346,104GT002,0,设计学类[205-8]班
1,202020212000347,104GT002,0,设计学类[201-4]班
2,202020212000557,104GT002,0,绘画[201-4]班
3,202020212000863,104GT002,0,管理科学与工程类[201-4]班
4,202020212001028,104GT002,0,新闻传播学类[201-3]班
...,...,...,...,...
289,202020212008325,780GL005,3,基础医学201班
290,202020212008885,780GL005,3,医学影像学201班
291,202020212008886,780GL005,3,医学影像学202班
292,202020212009052,780GL005,3,眼视光医学201班


# 主要函数内容

In [107]:
#功能描述：判断当前考场是否安排
#输入：教室名称，教室时间（--索引当前教室的状态）
#输出：当前教室是否安排，安排则为true
def classroom_isbz(cr_name,time_no):
    return (D[(D.cr_name==cr_name)&(D.mti_no==time_no)].bz==0)
#功能描述：判断当前时段的教室是否符合当前的课程，矛盾检测
#输入：教室名称（--求教室的人数），当前班级的人数
#输出：是否能够安排，能安排则为true，不能安排则
def student_num_fit(cr_names,cl_num):
    if (classroom_inf[classroom_inf.cr_name==cr_names].cr_testseat.values>=(cl_num)):
        return True
    else:
        return False

In [108]:
new_D

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,1,慧源楼101,前湖校区,1,1,None
1,2,慧源楼101,前湖校区,2,1,None
2,3,慧源楼101,前湖校区,3,1,None
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2419,2420,慧源楼520,前湖校区,20,1,None
2420,2421,慧源楼520,前湖校区,21,1,None
2421,2422,慧源楼520,前湖校区,22,1,None
2422,2423,慧源楼520,前湖校区,23,1,None


In [109]:
%%time
#功能描述：进行排考
#输入：教室时间笛卡儿乘积；班级编号与时间笛卡儿乘积
#输出：教室时间班级编号笛卡儿
def schedule_algorithm(new_D,temp):
    cannt_find_room_notice=[]
    for cl_no in temp.iloc[:,0].values:
        cl_num=get_class_student_number(cl_no)          #取出对应教室的学生数量
        fit_time=temp[temp['ci_nation_no']==cl_no]["test_time"].values[0] #取出第一个时间段
        fittime_room=D[D.mti_no==fit_time+1]                           #根据时间段找教室
        for room_name in fittime_room.cr_name.values.tolist():          #遍历教室
            if classroom_isbz(room_name,fit_time).values==False and student_num_fit(room_name,cl_num):
                new_D[(new_D.cr_name==room_name)&(new_D.mti_no==fit_time)]=new_D[(new_D.cr_name==room_name)&(new_D.mti_no==fit_time)].fillna(0)
                new_D.iloc[new_D[(new_D.cr_name==room_name)&(new_D.mti_no==fit_time)].index[0],0]=cl_no
                break
        else:
            cannt_find_room_notice.append(cl_no)
    return new_D,cannt_find_room_notice

Wall time: 0 ns


In [110]:
%%time
#alldata---所有考场的信息
#cannt_find_room---未安排的班级编号
#outdata-----已经安排好的考场情况
alldata,cannt_find_room=schedule_algorithm(new_D,temp) 
outdata=alldata[alldata.bz==0]

Wall time: 25.4 s


In [111]:
outdata

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,202020212003601,慧源楼101,前湖校区,1,1,0
1,202020212000466,慧源楼101,前湖校区,2,1,0
2,202020212001472,慧源楼101,前湖校区,3,1,0
24,202020212036230,慧源楼111,前湖校区,1,1,0
25,202020212008326,慧源楼111,前湖校区,2,1,0
26,202020212008325,慧源楼111,前湖校区,3,1,0
288,202020212003603,慧源楼125,前湖校区,1,1,0
289,202020212003772,慧源楼125,前湖校区,2,1,0
290,202020212003761,慧源楼125,前湖校区,3,1,0
312,202020212036197,慧源楼126,前湖校区,1,1,0


In [112]:
alldata

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,202020212003601,慧源楼101,前湖校区,1,1,0
1,202020212000466,慧源楼101,前湖校区,2,1,0
2,202020212001472,慧源楼101,前湖校区,3,1,0
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2419,2420,慧源楼520,前湖校区,20,1,None
2420,2421,慧源楼520,前湖校区,21,1,None
2421,2422,慧源楼520,前湖校区,22,1,None
2422,2423,慧源楼520,前湖校区,23,1,None


In [113]:
len(temp)

294

In [114]:
print(cannt_find_room[:5])
len(cannt_find_room)

[202020212000346, 202020212000347, 202020212000557, 202020212000863, 202020212001028]


242

In [115]:
outdata

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,202020212003601,慧源楼101,前湖校区,1,1,0
1,202020212000466,慧源楼101,前湖校区,2,1,0
2,202020212001472,慧源楼101,前湖校区,3,1,0
24,202020212036230,慧源楼111,前湖校区,1,1,0
25,202020212008326,慧源楼111,前湖校区,2,1,0
26,202020212008325,慧源楼111,前湖校区,3,1,0
288,202020212003603,慧源楼125,前湖校区,1,1,0
289,202020212003772,慧源楼125,前湖校区,2,1,0
290,202020212003761,慧源楼125,前湖校区,3,1,0
312,202020212036197,慧源楼126,前湖校区,1,1,0


In [116]:
temp

,ci_nation_no,ci_course_no,test_time,ci_class_name
0,202020212000346,104GT002,0,设计学类[205-8]班
1,202020212000347,104GT002,0,设计学类[201-4]班
2,202020212000557,104GT002,0,绘画[201-4]班
3,202020212000863,104GT002,0,管理科学与工程类[201-4]班
4,202020212001028,104GT002,0,新闻传播学类[201-3]班
...,...,...,...,...
289,202020212008325,780GL005,3,基础医学201班
290,202020212008885,780GL005,3,医学影像学201班
291,202020212008886,780GL005,3,医学影像学202班
292,202020212009052,780GL005,3,眼视光医学201班


In [ ]:
def get_outcsv(outdata):
    outdata=outdata.rename(columns={0:"ci_nation_no",1:"cr_name",2:"cr_area",3:"test_time",4:"cr_state",5:"bz"})
    pd.merge(temp,outdata,on=["ci_nation_no"]).iloc[:,[1,2,3,4]]

In [66]:
df=pd.merge(temp,outdata,on=["ci_nation_no"]).iloc[:,[1,2,3,4]]

In [68]:
#导出排好考场班级数据
df.to_csv("demo0.csv")

In [74]:
#导出未排好班级考场数据
temp[temp.apply(lambda x:x[0] in cannt_find_room,axis=1)].to_csv("demo1_canfind.csv")

# 初始数据分析--与前面算法没直接关系

In [13]:
classroom_inf.iloc[:,9].value_counts()    #----考场座位分布情况

38     37
30     36
36      8
58      6
62      5
80      3
48      3
100     1
94      1
68      1
Name: 9, dtype: int64